In [2]:
## Sample of Sepolia Network


from web3 import Web3
import hashlib
import yfinance as yf
import json

# --- Wallet setup ---
pKey   = '0x58cc6dda0b164c045b02b75ad8f3cb472645b9d863a83129a1f6ca560fd141c7'
wal_01 = '0x5afA5e8E6b31b6E57799cF4dF109Fd255fEb5Ab7'
wal_02 = '0x3a07279c76DbDb90833241AcFC7f32C54ee281eD'

# Connect to Sepolia via Infura
sepolia_rpc = "https://sepolia.infura.io/v3/fde74b6244e74502b31d2062bdc69d58"
w3 = Web3(Web3.HTTPProvider(sepolia_rpc))

# Check connection
if not w3.is_connected():
    raise Exception("Failed to connect to Sepolia")
print("✅ Connected to Sepolia Testnet")

# Set up sender from private key
sender = w3.eth.account.from_key(pKey)

# Receiver address
to_address = wal_02

# --- Load financial data ---
ticker     = 'AAPL'
start_date = '2024-01-01'
end_date   = '2025-03-31'

def load_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

df = load_data(ticker, start_date, end_date)
print(df.tail())

fileName = "aapl_data.json"
df.to_json(fileName, orient="split", date_format="iso")
print(f"✅ AAPL data saved to {fileName}")

# --- Read and hash file content ---
try:
    with open(fileName, "rb") as f:
        file_bytes = f.read()
        hash_hex = hashlib.sha256(file_bytes).hexdigest()
        print("🔐 SHA-256 hash of file:", hash_hex)
except FileNotFoundError:
    raise Exception(f"File '{fileName}' not found!")

# Convert hash to bytes for Ethereum tx data field
data_bytes = w3.to_bytes(hexstr=hash_hex)

# Get nonce for sender
nonce = w3.eth.get_transaction_count(sender.address)

# Build transaction
tx = {
    "nonce": nonce,
    "to": to_address,
    "value": w3.to_wei(0, "ether"),
    "gas": 50000,
    "gasPrice": w3.eth.gas_price,
    "data": data_bytes,
    "chainId": 11155111  # Sepolia chain ID
}

# Sign the transaction
signed_tx = w3.eth.account.sign_transaction(tx, pKey)

# Send the transaction
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

# Output the transaction hash
tx_hash_hex = w3.to_hex(tx_hash)
print(f"📤 Transaction sent! Hash: {tx_hash_hex}")



[*********************100%***********************]  1 of 1 completed

✅ Connected to Sepolia Testnet
Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2025-03-24  220.729996  221.479996  218.580002  221.000000  44299500
2025-03-25  223.750000  224.100006  220.080002  220.770004  34493600
2025-03-26  221.529999  225.020004  220.470001  223.509995  34466100
2025-03-27  223.850006  224.990005  220.559998  221.389999  37094800
2025-03-28  217.899994  223.809998  217.679993  221.669998  39818600
✅ AAPL data saved to aapl_data.json
🔐 SHA-256 hash of file: 6a649f2d9a2fb237bfdd1f635e9cd87edf4074a5ebe3a1446eb6be34bc6ef92e


📤 Transaction sent! Hash: 0x470070bd71440c2814fe0736e4c7d8b15260757c31aec4a7b09bd5ec9d9219a3



0. define the checkpoints - 5
1. empty dict with date/time and name (lstm_explanation_protocol)
2. we create the hash_hex of all the 5 data 
3. convert to data_bytes = w3.to_bytes(hexstr=hash_hex)
4. checkpoint name and tx_hash_hex values in lstm_explanation_protocol

